#YouTubeダウンロード  
参考:  
https://qlitre-weblog.com/pysimplegui-layout/
https://mebee.info/2022/02/04/post-34359/  
https://news.mynavi.jp/techplus/article/zeropython-73/

##レイアウトを作成

In [10]:
import PySimpleGUI as sg
from PIL import Image, ImageTk
import io
import youtube_dl

#先程確認して決めたテーマカラーを設定
sg.theme('Purple')

"""
公式のサンプル集に乗ってたコードをそのまま引用
maxsizeを大きくすると、大きな画像を読み込んだ際にGUIがその分大きくなる
今回はフレームサイズ以下になるように、450×450を表示最大サイズとした
"""

def get_img_data(f, maxsize=(450, 450), first=False):
    """画像を読み込む関数"""
    global status_text  # 画像サイズをGUI表示させるためにグローバル変数で関数外でも参照できるようにする
    img = Image.open(f)
    status_text = "%d x %d" % img.size  # オリジナルの画像サイズ
    img.thumbnail(maxsize)  # アスペクト比を維持しながら、指定したサイズ以下の画像に縮小
    status_text += " (%d x %d)" % img.size  # 縮小された画像サイズ
    if first:                     # tkinter is inactive the first time
        bio = io.BytesIO()
        img.save(bio, format="PNG")
        del img
        return bio.getvalue()
    return ImageTk.PhotoImage(img)


#動画をダウンロード
def MP4_DL(url, folder):

    #outtmpl = str(num) +'%(title)s.%(ext)s'
    outtmpl = '%(title)s.%(ext)s'
    down_dir = folder
    url = url

    ydl_opts = {
        #'format': 'bestvideo+bestaudio/best',
        'format': 'mp4',
        'outtmpl': down_dir + outtmpl
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

#mp3をダウンロードするとき
def MP3_DL(url, folder):
    #outtmpl = str(num) +'%(title)s.%(ext)s'
    outtmpl = '%(title)s.%(ext)s'
    down_dir = folder
    url = url

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': down_dir + outtmpl,
        'postprocessors': [
            {'key': 'FFmpegExtractAudio',
             'preferredcodec': 'mp3',
             'preferredquality': '192'},
            {'key': 'FFmpegMetadata'},
        ],
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def download_YouTube(mp4, mp3, url, folder):
    if mp4 == True:
        pass
    if mp3 == True:
        pass


#GUIへ初期画像を登録する(適当にパワポとかで作っておけばいい)
fname_first = './logo.png'
image_elem = sg.Image(data=get_img_data(fname_first, first=True))

#画像サイズを表示させる部分を変数化しておく(画像毎にアップデートさせるため)
status_elem = sg.Text(key='-STATUS-', size=(64, 1))

"""
・sg.Imageで画像部品をのせられる
・sg.Textでテキスト部品をのせられる
・sg.InputTextでテキスト入力エリアをのせる(画像Pathを表示させる部分)
・sg.FileBrowseでWindowsでよく見るファイル選択画面を出せる(InputTextの横に置けばtextを自動入力してくれる)
・sg.Submitはいわゆる「決定ボタン」。今回はOCR開始ボタンとして使った
・sg.MLineはテキスト出力エリア
・keyは、後でイベントを追加する時に参照する変数名
・後は省略できるが、サイズやフォントも各命令で指定出来る
"""
#フレーム1
frame1 = sg.Frame('',
                  [
                      #画像表示 ※初期画面では自分で用意した適当な画像を表示
                      [
                          image_elem,
                      ],
                      # テキストレイアウト
                      [
                          sg.Text('①ダウンロードしたいアドレスを入力してね（複数入力可）',
                                  font=('メイリオ', 18))
                      ],
                      #アドレス入力バー 
                      [sg.InputText('', key='-url1-')], [sg.],
                      [sg.InputText('', key='-url2-')],
                      [sg.InputText('', key='-url3-')],
                      [sg.InputText('', key='-url4-')],
                      [sg.InputText('', key='-url5-')],
                      # MP3 or MP4
                      [
                          sg.Text("②ダウンロード形式（複数選択可）",
                                  font=('メイリオ', 18)),
                      ],
                      # チェックボックス
                      [
                          sg.Checkbox("MP4（動画）", font=(
                              'メイリオ', 14), key='-mp4-', enable_events=True, default=False),
                          sg.Checkbox("MP3（音声）", font=(
                              'メイリオ', 14), key='-mp3-', enable_events=True, default=False),
                      ],
                      #保存フォルダ
                      [
                          sg.Text("③保存フォルダ選択",
                                  font=('メイリオ', 18)),
                      ],
                      [
                          sg.InputText(
                          ), sg.FolderBrowse(key="-folder-")
                      ],
                      #ダウンロード開始ボタン
                      [
                          sg.Submit(button_text='ダウンロード開始',
                                    font=('メイリオ', 18), size=(18, 3), pad=((10,10),(10,10)), key='button_download')
                      ],
                  ], size=(500, 700) #sizeは横-->縦の順
                  )

#フレーム2
frame2 = sg.Frame('',
                  [
                      # テキストレイアウト
                      [
                          sg.Text("ダウンロード進捗"),
                      ],
                      # MLineでテキストエリアを作成。sizeは「**列×**行」を表している
                      [
                          sg.MLine(font=('メイリオ', 8), size=(
                              50, 60), key='output'),
                      ]
                  ], size=(500, 700)
                  )

#左と右のフレームを合体させた全体レイアウト
layout = [
    [frame1, frame2] 
]

#GUIタイトルと全体レイアウトをのせたWindowを定義する
window = sg.Window('YouTube_download', layout, resizable=True)

#GUI表示実行部分
while True:
    # ウィンドウ表示
    event, values = window.read()

    #クローズボタンの処理
    if event is None:
        print('exit')
        break

    if event == "button_download":
        
        #ダウンロード先URLをリスト化
        url1 = values["-url1-"]
        url2 = values["-url2-"]
        url3 = values["-url3-"]
        url4 = values["-url4-"]
        url5 = values["-url5-"]
        url = [i for i in [url1, url2, url3, url4, url5] if i != ""]

        #どの項目をダウンロードするか（True or False)
        mp3 = values["-mp3-"]
        mp4 = values["-mp4-"]

        #保存先フォルダ
        folder = values["-folder-"]

        if mp4 is True:
            MP4_DL(url, folder)
        
        if mp3 is True:
            MP3_DL(url, folder)
        pass

window.close()


exit
